In [1]:
import os
import subprocess
import sys
import glob
sys.path.append('../sdpm_py_util')
sys.path.append('../web_util')
import plotting_functions as pltfuns
import netCDF4 as nc
import web_functions as webfuns
import ocn_functions as ocnfuns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta


/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/ocn_functions.py:34: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


In [6]:
def get_longest_forecast():
    # this return the forecast start time and the forecast last time
    # ftime is a string, tmx_tot is a datetime object
    # and is the longest forecast we can make given the hycom data 
    # we have

    # get the unique dates of the hycom forecasts
    t0s = ocnfuns.stored_hycom_dates()

    # get all .nc files in the hycom directory
    matched_files = glob.glob('/scratch/PFM_Simulations/hycom_data/*.nc')
    fnms = []
    for mf in matched_files:
        _, tail = os.path.split(mf)
        fnms.append(tail)

    # DB is the dictionary that stores all of the time stamps of each .nc file
    DB = dict()
    # we loop through the variables
    vars = ['ssh','t3z','s3z','u3z','v3z']
    # t0s and the file name strings for time don't match, make 
    # the formatting match
    t0su = []
    for t0 in t0s:
        t0su.append(t0+'T12')

    # initialize the dictionary with empty lists for each forecast t0
    # and each variable
    for var in vars:
        for t0 in t0su:
            DB[(var,t0)]=[]

    # now we loop through all the files and keep track of the time stamps
    # for each var and forecast time
    for fn in fnms:
        var = fn[3:6]
        t0  = fn[7:20]
        tf  = fn[24:37]
        DB[(var,t0)].append(tf)

    # set up a dictionary of maximum times, as a function of var and 
    # forecast
    TMX = dict()
    for var in vars: # loop through variables
        if var == 'ssh':
            DT0 = 1  # set DT0, how space the times are for each var
            NT = 193 # the number of files if it is a full forecast
        else:
            DT0 = 3
            NT = 65
        for t0 in t0su: # loop through the forecasts
            tfs = DB[(var,t0)]
            tfdtl = []
            for tf in tfs:
                tfdt = datetime.strptime(tf,'%Y-%m-%dT%H')
                tfdtl.append(tfdt)
            
            # there are a couple of options, no data, full forecast, and partial
            if len(tfdtl) == 0:
                # there is no data
                TMX[(var,t0)] = datetime.strptime(t0,'%Y-%m-%dT%H')
            elif len(tfdtl) == NT:
                # we have all the data, full forecast!
                TMX[(var,t0)] = np.max(tfdtl)
            else:
                # a parital forecast. 2 options here. we skip times
                tfdta = np.array(tfdtl)
                tfdta_s = np.sort(tfdta)
                for aa in np.arange(len(tfdta_s)-1):
                    dt = tfdta_s[aa+1]-tfdta_s[aa]
                    dt_hr = int( dt.total_seconds()/3600 )
                    if dt_hr > DT0: # if we get here, files are skipped
                        TMX[(var,t0)] = tfdta_s[aa]
                        break 
                # if we dont skip files, this is TMX    
                TMX[(var,t0)] = tfdta_s[aa+1]
                
    # set up a dictionary to figure out what the longest forecast we can do 
    # this is a function of forecast start time
    tmxx = dict()
    for t0 in t0su:
        tmxx[t0] = []
        tmx_tot = datetime.strptime('2100','%Y') # a dummy to start with
        for var in vars:
            tmx2 = TMX[(var,t0)]
            if tmx2 < tmx_tot:
                tmx_tot = tmx2
        tmxx[t0] = tmx_tot # this is now the maximum forecast date
                           # for the forecast t0

    # we now determine which forecast t0 has the larges tmx. what we want
    tmx_tot = datetime.strptime('2000','%Y') # initialize this
    for t0 in t0su: # t0 is sorted
        print(t0)
        tmxx3 = tmxx[t0]
        print(tmxx3)
        if tmxx3 >= tmx_tot: # this insures we use the latest forecast
            tmx_tot = tmxx3
            ftime = t0

    # return the forecast start time and the forecast last time
    # ftime is a string, tmx_tot is a datetime object
    return ftime, tmx_tot


In [7]:

ftime,tmx_tot = get_longest_forecast()
print('we can use the hycom forecast from')
print(ftime)
print('to do a forecast to')
print(tmx_tot)
print('this is the best we can do.')
#print(ftime[0:4]+ftime[5:7]+ftime[8:10])


2025-04-26T12
2025-05-04 12:00:00
2025-04-27T12
2025-04-27 12:00:00
we can use the hycom forecast from
2025-04-26T12
to do a forecast to
2025-05-04 12:00:00
this is the best we can do.
